# Shortest Path Algorithms

We survey algorithms solving the shortest path problem:

$$
\begin{alignat*}{3}
& \text{SP} \quad=\quad
    && \text{minimize}   \quad && c^T x \\
&   && \text{subject to} \quad && A x = d \\
&   &&                         && x \geq 0 \\
\end{alignat*}
$$

In [1]:
using DataStructures: BinaryMinHeap, DefaultDict
using BenchmarkTools

In [2]:
struct Arc
    length::Float64
    cost::Float64
end

function sp(adj::Dict{Int, Dict{Int, Arc}}, s::Int, t::Int)
    frontier = BinaryMinHeap{Tuple{Float64, Int, Int}}()
    shortest = DefaultDict{Int, Float64}(Inf)
    parent = Dict{Int, Int}()

    push!(frontier, (0.0, s, s))
    while !isempty(frontier)
        cost, node, pred = pop!(frontier)

        if haskey(shortest, node)
            continue
        end

        parent[node] = pred
        shortest[node] = cost

        if node == t
            return cost
        else
            for (adjacent, edge) in adj[node]
                push!(frontier, (cost + edge.cost, adjacent, node))
            end
        end
    end

    return missing
end

sp (generic function with 1 method)

In [24]:
function csp(adj::Dict{Int, Dict{Int, Arc}}, s::Int, t::Int, R::Float64 = Inf)
    frontier = BinaryMinHeap{Tuple{Float64, Float64, Int, Int}}()
    shortest = DefaultDict{Tuple{Int, Float64}, Float64}(Inf)
    parent = Dict{Tuple{Int, Float64}, Tuple{Int, Float64}}()

    push!(frontier, (0.0, 0.0, s, -1))
    while !isempty(frontier)
        cost, distance, node, pred = pop!(frontier)

        if haskey(shortest, (node, distance))
            continue
        end

        if pred != -1
            parent[node, distance] = (pred, distance - adj[pred][node].length)
        end
        shortest[node, distance] = cost

        if node == t
            return cost
        else
            for (adjacent, edge) in adj[node]
                if distance + edge.length <= R
                    push!(frontier, (cost + edge.cost, distance + edge.length, adjacent, node))
                end
            end
        end
    end

    return missing
end

csp (generic function with 2 methods)

In [33]:
n = 90

adj = Dict{Int, Dict{Int, Arc}}()

function ok(i::Int, j::Int, n::Int)
    0 <= i < n && 0 <= j < n
end

for k in 0:(n * n - 1)
    i, j = divrem(k, n)

    ns = adj[k] = Dict{Int, Arc}()
    for (ni, nj) in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
        if ok(ni, nj, n)
            ns[n * ni + nj] = Arc(1, rand(1:5))
        end
    end
end

In [34]:
s = rand(0:(n * n - 1))
t = rand(0:(n * n - 1))
sp(adj, s, t), csp(adj, s, t, 1000000.0)

(166.0, 166.0)

In [35]:
@benchmark begin
    s = rand(0:(n * n - 1))
    t = rand(0:(n * n - 1))
    sp(adj, s, t)
end

BenchmarkTools.Trial: 1655 samples with 1 evaluation.
 Range (min … max):  1.166 μs … 7.515 ms  ┊ GC (min … max): 0.00% … 38.01%
 Time  (median):     2.940 ms             ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.018 ms ± 1.849 ms  ┊ GC (mean ± σ):  0.88% ±  5.37%

  ▆▅▃ ▅▁▄▁▂█▁ ▃▂▂ ▃  ▂▇▆  ▄▃▄ ▃▁ ▆ ▄▁▂     ▂▁▅▃ ▃ ▁▂ ▃▃▁     
  █████████████████▇▄███▆▇███▇████████▇█▇██████▆█▆██▇████▆▆ ▆
  1.17 μs        Histogram: frequency by time       6.31 ms <

 Memory estimate: 1.48 KiB, allocs estimate: 18.

In [37]:
@benchmark begin
    s = rand(0:(n * n - 1))
    t = rand(0:(n * n - 1))
    csp(adj, s, t, 45.0)
end

BenchmarkTools.Trial: 207 samples with 1 evaluation.
 Range (min … max):  55.334 μs … 40.288 ms  ┊ GC (min … max): 0.00% … 3.60%
 Time  (median):     25.698 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.267 ms ± 10.604 ms  ┊ GC (mean ± σ):  0.70% ± 3.41%

                      ▁  ▁   ▁  ▂  ▁▂ ▁▄▄▁  ▄  ▂ ▁ ▄▇ ▅   ▅█   
  █▃▃▅▅█▁▆▅▃▆▁▁▃▃▅▅▁█▅█▅▆█▃▁▆█▅▃█▆███▃█████▆████▆█▆██▆███▃██▃ ▅
  55.3 μs         Histogram: frequency by time        39.4 ms <

 Memory estimate: 50.14 KiB, allocs estimate: 33.